# Challenge 3

El lab lo he hecho con una muestra de 500 porque con 20000 me acabó petando el ordenador y tuve que reiniciarlo.

In [15]:
import pandas as pd

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import *
from nltk.stem.porter import *
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from stop_words import get_stop_words

import re

In [4]:
tweer_data = pd.read_csv('./Sentiment140.csv')

In [5]:
tweer_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
def clean_up(s):
    s = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', s)
    s = re.sub('[^a-zA-Z_]+', ' ', s)
    s = s.lower()
    return s

def tokenize(s):
    return word_tokenize(s)

def stem_and_lemmatize(l):
    result = []
    for i in l:
        sno = nltk.stem.SnowballStemmer('english')
        x = sno.stem(i)
        lemmatizer = WordNetLemmatizer()
        x = lemmatizer.lemmatize(x)
        result.append(x)
    return result

def remove_stopwords(l):
    stop_words = get_stop_words('en')
    filtered_words = [word for word in l if word not in stop_words]
    return filtered_words

In [7]:
sample = tweer_data.sample(n=500, random_state=1)

In [8]:
sample['text_processed'] = sample['text'].apply(clean_up)
sample['text_processed'] = sample['text_processed'].apply(tokenize)
sample['text_processed'] = sample['text_processed'].apply(stem_and_lemmatize)
sample['text_processed'] = sample['text_processed'].apply(remove_stopwords)

In [9]:
sample.head()

,target,id,date,flag,user,text,text_processed
514293,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there ...,"[miss, nikki, nu, nu, alreadi, shes, alway, ne..."
142282,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,So I had a dream last night. I remember a sig...,"[dream, last, night, rememb, sign, clear, told..."
403727,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,@girlyghost ohh poor sickly you (((hugs)) ho...,"[girlyghost, ohh, poor, sick, hug, hope, feel,..."
649503,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again,[rain]
610789,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,@MissKeriBaby wish I was in LA right now,"[misskeribabi, wish, wa, la, right, now]"


In [10]:
words = []
for x in sample.text_processed:
    words += x
words[:10]

['miss', 'nikki', 'nu', 'nu', 'alreadi', 'shes', 'alway', 'need', 'thank', 'u']

In [11]:
fdist = FreqDist(words)
top_words = sorted(fdist, key=fdist.get, reverse=True)[:5000]

# fdist = FreqDist(lista)
# voc = fdist.most_common(5000)
# top = [x[0] for x in voc]

In [16]:
def find_features(document):
    words = set(document)
    features = {}
    for w in top_words:
        features[w] = (w in words)
    sentiment = SentimentIntensityAnalyzer().polarity_scores(" ".join(document))
    if sentiment["pos"] > 0.2:
        sentiment = True
    else:
        sentiment = False
    return (features, sentiment)

In [17]:
features_in_text = sample['text_processed'].apply(find_features)

In [20]:
train_set, test_set = features_in_text[:400], features_in_text[400:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [23]:
classifier.show_most_informative_features()

Most Informative Features
                    love = True             True : False  =     18.2 : 1.0
                     lol = True             True : False  =      8.1 : 1.0
                    good = True             True : False  =      6.9 : 1.0
                    like = True             True : False  =      6.4 : 1.0
                    look = True             True : False  =      5.5 : 1.0
                   watch = True            False : True   =      4.5 : 1.0
                    next = True            False : True   =      3.9 : 1.0
                       p = True             True : False  =      3.8 : 1.0
                   alway = True             True : False  =      3.8 : 1.0
                  wonder = True             True : False  =      3.8 : 1.0


In [24]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 67.0
